# Artikelbasiertes, kollaboratives Filtern

Zuerst stellen wir wieder sicher, dass das MovieLens 100k Datenset heruntergeladen ist:

In [52]:
import urllib.request
import zipfile
import io

movieLensData = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
with urllib.request.urlopen(movieLensData) as f:
    zf = open('ml-100k.zip', 'wb')
    zf.write(f.read())
    zf.close()
    with zipfile.ZipFile('ml-100k.zip') as zipFileObj:
        zipFileObj.extractall(".")

Anschließend lesen wir diesen Datensatz in ein DataFrame von Pandas ein. Und fügen gleichzeitig noch einen extra Benutzer mit der ID 0 zu den Daten hinzu (extra_data). Den werden wir später noch brauchen:

In [53]:
import urllib.request
import zipfile
import io

import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('./ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3), encoding="ISO-8859-1")

m_cols = ['movie_id', 'title']
movies = pd.read_csv('./ml-100k/u.item', sep='|', names=m_cols, usecols=range(2), encoding="ISO-8859-1")

extra_data = [[0, 56, 5], [0, 42, 3], [0, 50, 5],[0,172,3],[0,204,3], [0,181,4], [0,231,5],[0,174,4]]
ratings = ratings.append(pd.DataFrame(extra_data, columns=r_cols), ignore_index=True)

ratings = pd.merge(movies, ratings)

ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [54]:
movies.head(100)

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
6,7,Twelve Monkeys (1995)
7,8,Babe (1995)
8,9,Dead Man Walking (1995)
9,10,Richard III (1995)


Auf Basis dieser Tabellen konstruiert man jetzt eine Pivot-Tabelle wo zu jedem Benutzer angeben ist, welche Filme er bewertet hat. NaN bedeutet, dass zu diesem Eintrag keine Daten existieren bzw. der Nutzer den Film nicht angeschaut hat. 

In [55]:
userRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
userRatings.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Jetzt wird gezaubert! Mit Hilfe der corr() - Methode wird zu jedem Film ein Korrelationswert zu jedem Spaltenpaar berechnet. Heraus kommt eine Tabelle wo zu jedem Film die Korrelation zu jedem anderen Film berechnet ist, wo also mindestens ein Nutzer beide Filme bewertet hat. 

Hat kein Nutzer ein Filmepaar bewertet, steht in der Tabelle ein NaN. Cool, dass das so einfach geklappt hat!

In [56]:
corrMatrix = userRatings.corr()
corrMatrix.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),1.0,NaN,-1.000000,-0.500000,-0.500000,0.522233,NaN,-0.426401,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-900 (1994),NaN,1.0,NaN,NaN,NaN,NaN,NaN,-0.981981,NaN,NaN,...,NaN,NaN,NaN,-0.944911,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),-1.0,NaN,1.000000,-0.049890,0.269191,0.048973,0.266928,-0.043407,NaN,0.111111,...,NaN,-1.000000,NaN,0.158840,0.119234,0.680414,0.000000,0.707107,NaN,NaN
12 Angry Men (1957),-0.5,NaN,-0.049890,1.000000,0.666667,0.256625,0.274772,0.178848,NaN,0.457176,...,NaN,NaN,NaN,0.096546,0.068944,-0.361961,0.144338,1.000000,1.0,NaN
187 (1997),-0.5,NaN,0.269191,0.666667,1.000000,0.596644,NaN,-0.554700,NaN,1.000000,...,NaN,0.866025,NaN,0.455233,-0.500000,0.500000,0.475327,NaN,NaN,NaN


Wenn jetzt aber nur ein Nutzer ein Filmepaar bewertet hat, schleicht sich sehr leicht ein Fehler ein. Daher beschränken wir die Korrelation auf die Filme, die von vielen Personen bewertet wurden. 

Dadurch bekommen wir einerseits populärere Filme als Ergebnis (weil nur diese von hinreichend vielen Nutzern bewertet wurden), können uns aber auch "sicherer" sein, dass das Ergebnis stimmt. 

Um dies zu machen, setzen wir den Parameter "min_periods" auf 100, d.h. mindestens 100 Nutzer müssen ein entsprechendes Filmepaar bewertet haben:

In [57]:
corrMatrix = userRatings.corr(method='pearson', min_periods=100)
corrMatrix.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-900 (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187 (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Erinnerst du dich noch an den Nutzer den wir am Anfang mit in unsere Daten geschrieben haben? Dieser Nutzer hatte die ID 0, und mag Star Wars, sowie The Empire Strikes Back, mochte aber den Film "Gone with the Wind" überhaupt nicht. 

Schauen wir uns nochmal die Filme an, die dieser Nutzer bewertet hat. Dafür kann die `.loc[0]` - Schreibweise verwendet werden, heraus kommt eine Liste aller Filme mit der jeweiligen Bewertung - oder NaN sollte keine Bewertung existieren. Die NaN - Werte müssen also noch herausgefiltert werden, dies geschieht mit der Methode `dropna()`:

In [58]:
myRatings = userRatings.loc[0].dropna()
myRatings

title
Back to the Future (1985)          3.0
Batman Returns (1992)              5.0
Clerks (1994)                      3.0
Empire Strikes Back, The (1980)    3.0
Pulp Fiction (1994)                5.0
Raiders of the Lost Ark (1981)     4.0
Return of the Jedi (1983)          4.0
Star Wars (1977)                   5.0
Name: 0, dtype: float64

Jetzt gehen wir jeden der 3 bewerteten Filme durch, und erstellen für jeden Film Empfehlungen. Damit diese Empfehlungen möglichst solche Filme enthalten, die ähnlich zu den Filmen sind, die ich gut bewertet habe, wird der Ähnlichkeitswert mit meiner Bewertung multipliziert:

In [59]:
simCandidates = pd.Series()
for i in range(0, len(myRatings.index)):
    print("Füge Ähnlichkeiten für " + str(myRatings.index[i]) + " hinzu...")
    # Berechne Filme, die ähnlich sind zu den Filmen die ich bewertet habe
    sims = corrMatrix[myRatings.index[i]].dropna()
    # Multipliziere den Ähnlichkeitswert mit meiner Bewertung
    sims = sims.map(lambda x: x * myRatings[i])
    # Und füge diesen Eintrag zur Liste hinzu
    simCandidates = simCandidates.append(sims)
    
#Glance at our results so far:
print("sortieren...")
simCandidates.sort_values(inplace = True, ascending = False)
print(simCandidates.head(10))

Füge Ähnlichkeiten für Back to the Future (1985) hinzu...
Füge Ähnlichkeiten für Batman Returns (1992) hinzu...
Füge Ähnlichkeiten für Clerks (1994) hinzu...
Füge Ähnlichkeiten für Empire Strikes Back, The (1980) hinzu...
Füge Ähnlichkeiten für Pulp Fiction (1994) hinzu...
Füge Ähnlichkeiten für Raiders of the Lost Ark (1981) hinzu...
Füge Ähnlichkeiten für Return of the Jedi (1983) hinzu...
Füge Ähnlichkeiten für Star Wars (1977) hinzu...
sortieren...
Star Wars (1977)                   5.000000
Batman Returns (1992)              5.000000
Pulp Fiction (1994)                5.000000
Return of the Jedi (1983)          4.000000
Raiders of the Lost Ark (1981)     4.000000
Empire Strikes Back, The (1980)    3.715581
Return of the Jedi (1983)          3.361005
Clerks (1994)                      3.000000
Empire Strikes Back, The (1980)    3.000000
Back to the Future (1985)          3.000000
dtype: float64


Das sieht doch schonmal gut aus. Jetzt sind aber einige der Filme noch doppelt in unseren Ergebnissen vorhanden. Daher brauchen wir noch eine Gruppierung nach dem Namen des Filmes:

In [60]:
simCandidates = simCandidates.groupby(simCandidates.index).sum()

In [61]:
simCandidates.sort_values(inplace = True, ascending = False)
simCandidates.head(20)

Empire Strikes Back, The (1980)              13.733510
Star Wars (1977)                             13.170635
Return of the Jedi (1983)                    13.079754
Raiders of the Lost Ark (1981)               12.455985
Back to the Future (1985)                    11.547421
Indiana Jones and the Last Crusade (1989)    10.110891
Batman (1989)                                 9.573139
Batman Returns (1992)                         8.614231
Terminator 2: Judgment Day (1991)             8.552079
True Lies (1994)                              7.957528
Terminator, The (1984)                        7.572561
Field of Dreams (1989)                        7.529177
Abyss, The (1989)                             7.246395
Fugitive, The (1993)                          7.226762
Star Trek: The Wrath of Khan (1982)           7.152821
Firm, The (1993)                              7.148383
Braveheart (1995)                             7.103651
Jurassic Park (1993)                          6.914719
Pulp Ficti

Zu guter Letzt müssen wir noch die Filme entfernen, die ich schon bewertet habe - diese mir erneut vorzuschlagen macht vermutlich keinen Sinn:

In [62]:
filteredSims = simCandidates.drop(myRatings.index)
filteredSims.head(20)

Indiana Jones and the Last Crusade (1989)    10.110891
Batman (1989)                                 9.573139
Terminator 2: Judgment Day (1991)             8.552079
True Lies (1994)                              7.957528
Terminator, The (1984)                        7.572561
Field of Dreams (1989)                        7.529177
Abyss, The (1989)                             7.246395
Fugitive, The (1993)                          7.226762
Star Trek: The Wrath of Khan (1982)           7.152821
Firm, The (1993)                              7.148383
Braveheart (1995)                             7.103651
Jurassic Park (1993)                          6.914719
E.T. the Extra-Terrestrial (1982)             6.557887
Aliens (1986)                                 6.436100
Star Trek: First Contact (1996)               6.371103
Star Trek IV: The Voyage Home (1986)          6.258960
Jaws (1975)                                   6.230167
Usual Suspects, The (1995)                    6.210784
Top Gun (1

Fertig!

## Aufgabe

Wie ließen sich diese Ergebnisse verbessern? Liefert eine andere Methode zur Berechnung der Korrelation oder ein anderer Mindestwert bei `min_periods` bessere Ergebnisse? 

Es scheinen auch noch ein paar Filme durchgerutscht zu sein, die ähnlich zu "Gone with the Wind" sind, diesen habe ich ja sehr schlecht bewertet. Vielleicht sollten Filme, die ähnlich zum Film "Gone with the Wind" sind, abgestraft werden? 

Wenn du noch mehr coden möchtest: Wir berechnen hier ein Ergebnis, aber eigentlich könnten wir auch train/test verwenden, und den Algorithmus mit bekannten Daten überprüfen, z.B. auf Basis der Filme die ein Nutzer bereits angeschaut hat. Aber ob das dann wirklich zu "guten" Ergebnissen führt, darüber kann man natürlich diskutieren ;-) 